In [51]:
# PLACE ALL IMPORTS IN THIS CELL

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import brown

In [55]:
d = enchant.Dict("en_US")

In [6]:
# SET UP DATA

# convert csv to a pandas DataFrame format
train_df = pd.DataFrame.from_csv('data/train.csv')
test_df = pd.DataFrame.from_csv('data/test.csv')

In [3]:
# BASIC DATASET INFORMATION

# -- our train set is 1089 x 17
print(train_df.shape)
# -- our test set is 300 x 15 (the 2 missing columns are the label and a redundant field that resembles label)
print(test_df.shape)

(1089, 17)
(300, 15)


In [9]:
train_df.shape[0]

1089

In [7]:
# FIGURE OUT LABEL DISTRIBUTION [~57% of dataset came from Android]

# create dataset with only positive labels -- 619 positive labels in training set
df_train_pos = train_df[train_df.label == 1]
# create dataset with only negative labels -- 470 negative labels in training set
df_train_neg = train_df[train_df.label == -1]
print("Number of positive [Android] training points: " + str(len(df_train_pos)))
print("Number of negative [iPhone] training points: " + str(len(df_train_neg)))
print("Percentage of training points from Android: " + str(len(df_train_pos)/train_df.shape[0]))

Number of positive [Android] training points: 619
Number of negative [iPhone] training points: 470
Percentage of training points from Android: 0


In [10]:
# EXPLORE TIME FEATURE
print(train_df['created'])

id
0         7/12/16 0:56
1        7/11/16 22:18
2        7/11/16 21:40
3        7/11/16 19:51
4        7/11/16 11:57
5        7/10/16 18:58
6        7/10/16 18:42
7        7/10/16 18:27
8        7/10/16 12:02
9         7/9/16 21:22
10        7/9/16 15:18
11        7/8/16 23:26
12        7/8/16 21:31
13        7/8/16 14:32
14        7/8/16 13:20
15        7/8/16 11:02
16         7/8/16 1:52
17        7/7/16 20:09
18        7/7/16 20:06
19        7/7/16 20:04
20        7/7/16 18:07
21        7/7/16 11:33
22         7/7/16 1:34
23        7/6/16 21:58
24        7/6/16 14:06
25        7/6/16 14:01
26        7/6/16 13:24
27        7/6/16 13:12
28        7/6/16 13:08
29        7/6/16 12:31
             ...      
1059     1/16/16 19:32
1060     1/16/16 18:31
1061     1/16/16 18:26
1062     1/16/16 18:23
1063     1/16/16 18:22
1064     1/16/16 18:21
1065     1/16/16 16:11
1066     1/16/16 13:42
1067     1/16/16 13:31
1068      1/8/16 23:12
1069      1/8/16 22:36
1070      1/8/16 22:09
1071    

In [30]:
# DROPPING IRRELEVANT COLUMNS
train_df = train_df.drop(['favorited', 'truncated', 'replyToSID', 'id.1', 'replyToUID', 'statusSource', 'screenName', 'isRetweet', 'retweeted','longitude', 'latitude'], axis=1)


In [33]:
"@hello" in "@hello my name is"

True

In [54]:
d.check("Supreme")

True

In [64]:
# CREATING FEATURES 

#if tweet starts with quote
quotes = [t[0] == '"' for t in train_df['text']]
train_df['in_quotes'] = quotes

#it tweet contains @realdonaldtrump
train_df['uses_own_handle'] = ["@realDonaldTrump" in t for t in train_df['text']]

#if tweet contains http
train_df['contains_http'] = ["http" in t for t in train_df['text']]

#if tweet contains hashtag
train_df['contains_hashtag'] = ["#" in t for t in train_df['text']]

#check misspellings
word_set = set(brown.words())
def spell_check_sentence(sentence):
    misspelled = [not d.check(x) if x!= "" else False for x in sentence.split(" ")]
    return sum(misspelled)

train_df['num_of_misspellings'] = [spell_check_sentence(s) for s in train_df['text']]



In [46]:
a = [x for x in "Hello my name is".split(" ")]

In [48]:
sum([False, True])

1

In [76]:
train_df.apply(lambda row: row.label == 1 and 'MAGA' in row.text, axis=1).sum()

3

In [28]:
train_df['in_quotes'] = quotes

In [62]:
[d.check(x) for x in train_df['text'][0].split(" ")]

[True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False]

In [65]:
train_df

,text,favoriteCount,replyToSN,created,retweetCount,label,in_quotes,uses_own_handle,contains_http,contains_hashtag,num_of_misspellings
id,,,,,,,,,,,
0,Senior United States District Judge Robert E. ...,14207,NaN,7/12/16 0:56,5256,-1,False,False,True,False,2
1,Speech on Veterans' Reform: https://t.co/XB7R...,9666,NaN,7/11/16 22:18,3432,-1,False,False,True,False,3
2,Great poll- Florida! Thank you! https://t.co/4...,25531,NaN,7/11/16 21:40,8810,-1,False,False,True,False,4
3,Thoughts and prayers with the victims; and the...,28850,NaN,7/11/16 19:51,9112,-1,False,False,False,False,6
4,Join me in Westfield; Indiana- tomorrow night ...,12567,NaN,7/11/16 11:57,4144,-1,False,False,True,True,7
5,I heard that the underachieving John King of @...,22978,NaN,7/10/16 18:58,6564,1,False,False,False,False,4
6,The media is so dishonest. If I make a stateme...,44600,NaN,7/10/16 18:42,14520,1,False,False,False,False,4
7,President Obama thinks the nation is not as di...,35167,NaN,7/10/16 18:27,11975,1,False,False,False,False,2
8,Look what is happening to our country under th...,55495,NaN,7/10/16 12:02,19030,1,False,False,False,False,2
